In [89]:
# Importing Libraries for analysis

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 91, Finished, Available, Finished)

In [90]:
# Retrieving the tables for analysis
df_rides = spark.sql("SELECT * FROM test_lakehouse.rides")

df_shifts = spark.sql("SELECT * FROM test_lakehouse.shifts LIMIT 3")

df_locations = spark.sql("SELECT * FROM test_lakehouse.locations")


StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 92, Finished, Available, Finished)

In [91]:
pd_df_rides = df_rides.toPandas()
pd_df_shifts = df_shifts.toPandas()
pd_df_locations = df_locations.toPandas()


StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 93, Finished, Available, Finished)

In [95]:
# Eliminating all null field
pd_df_rides = pd_df_rides[pd_df_rides['dropoff_time'].notna()]


StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 97, Finished, Available, Finished)

In [96]:
pd_df_rides_with_origin = pd_df_rides.merge(
    pd_df_locations,
    left_on='origin_id',
    right_on='location_id',
    how='left')



StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 98, Finished, Available, Finished)

In [97]:
selecting_required_columns  =[ 'ride_id','pickup_time', 'dropoff_time', 'origin_id', 'destination_id','location_name', 'longitude', 'latitude']
# pd_df_rides_with_origin[[selecting_required_columns]]

pd_df_rides_with_origin  = pd_df_rides_with_origin[selecting_required_columns]



StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 99, Finished, Available, Finished)

In [98]:
pd_df_rides_with_origin.columns =  ['ride_id', 'pickup_time', 'dropoff_time', 'origin_id', 'destination_id',
       'location_name', 'origin_longitude', 'origin_latitude']

StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 100, Finished, Available, Finished)

In [99]:
#joining the latitute and longitude of the 
pd_df_rides_with_locations = pd_df_rides_with_origin.merge(
    pd_df_locations[['location_id','longitude', 'latitude']],
    left_on = 'destination_id',
    right_on = 'location_id',
    how= 'left')

StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 101, Finished, Available, Finished)

In [100]:
pd_df_rides_with_locations['dropoff_time'] = pd.to_datetime(pd_df_rides_with_locations['dropoff_time'])
pd_df_rides_with_locations['pickup_time'] = pd.to_datetime(pd_df_rides_with_locations['pickup_time'])

# Computing the Duration
pd_df_rides_with_locations['travel_duration_minutes']  = (pd_df_rides_with_locations['dropoff_time']-pd_df_rides_with_locations['pickup_time']).dt.total_seconds()/60

StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 102, Finished, Available, Finished)

In [102]:
pd_df_rides_with_locations.columns = ['ride_id', 'pickup_time', 'dropoff_time', 'origin_id', 'destination_id',
       'location_name', 'origin_longitude', 'origin_latitude', 'location_id',
       'destination_longitude', 'destination_latitude', 'travel_duration_minutes']

StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 104, Finished, Available, Finished)

In [103]:
pd_df_rides_with_locations.head()

StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 105, Finished, Available, Finished)

,ride_id,pickup_time,dropoff_time,origin_id,destination_id,location_name,origin_longitude,origin_latitude,location_id,destination_longitude,destination_latitude,travel_duration_minutes
0,1b867cb5-61c2-4439-ac7b-ab0eca2c8021,2025-04-01 15:20:00,2025-04-01 15:27:00,451048bb-f8bc-4ed7-8310-249edebc0f15,d9efad18-c918-4c96-9518-588bf2b84aa2,Ottawa / Hoffman,-80.47953,43.43196,d9efad18-c918-4c96-9518-588bf2b84aa2,-80.49364,43.45496,7.0
1,1cd9992b-9043-4e71-9f4f-999598116f30,2025-04-01 18:21:00,2025-04-01 18:31:00,451048bb-f8bc-4ed7-8310-249edebc0f15,9430dbd7-f075-42f0-8d74-c4d4e4ca718a,Ottawa / Hoffman,-80.47953,43.43196,9430dbd7-f075-42f0-8d74-c4d4e4ca718a,-80.52330,43.46210,10.0
2,22c32e75-5b08-4330-ad11-6e5368904e44,2025-04-01 19:01:00,2025-04-01 19:14:00,95ba0c99-3e86-4886-916e-97fa9c191513,39c36c68-a90f-418d-bc86-aca33132b96d,"53 Creek Ridge St, Kitchener, ON",-80.50125,43.38053,39c36c68-a90f-418d-bc86-aca33132b96d,-80.57073,43.44609,13.0
3,43d54607-418e-41aa-9aea-784bdfacb321,2025-04-02 00:16:00,2025-04-02 00:23:00,999f4d52-419c-48d2-a439-75f6f17dd62c,07285951-c0c5-4528-8512-804ded890402,Ottawa / Hoffman,-80.47950,43.43144,07285951-c0c5-4528-8512-804ded890402,-80.43488,43.42926,7.0
4,50026009-7c68-47fd-a756-ead93c36b61c,2025-04-02 03:20:00,2025-04-02 03:30:00,a698d7e0-bf82-42cb-aae6-daf9dbe44dea,51a9c1fe-c4ef-44c2-9d5e-31e293158764,Wellington / Moore,-80.50109,43.45509,51a9c1fe-c4ef-44c2-9d5e-31e293158764,-80.52825,43.49754,10.0


In [104]:


# Selecting features for training

selected_features_model = ['ride_id', 'origin_longitude', 'origin_latitude',
       'destination_longitude', 'destination_latitude',
       'travel_duration_minutes']
pd_df_rides_with_locations = pd_df_rides_with_locations[selected_features_model]


StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 106, Finished, Available, Finished)

In [107]:
# Building Linear model
x = pd_df_rides_with_locations[['origin_longitude', 'origin_latitude',
       'destination_longitude', 'destination_latitude']]

y = pd_df_rides_with_locations['travel_duration_minutes']


# Building training and test dataset for model building
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2)

lr = LinearRegression()
lr.fit(x_train, y_train)

# Getting the prediction 
y_pred = lr.predict(x_test)


StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 109, Finished, Available, Finished)

In [113]:
# model_evaluation
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

r2 = r2_score(y_test, y_pred)

print(f'r2 score for model is :{r2}')

StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 115, Finished, Available, Finished)

r2 score for model is :0.3154351591409802


In [123]:
coefficients =  lr.coef_

StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 125, Finished, Available, Finished)

In [124]:
intercept_model = lr.intercept_

intercept_model

StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 126, Finished, Available, Finished)

6686.524910742116

In [132]:
new_table = pd.DataFrame({
    'intercept':intercept_model,
    'origin_longitude':[coefficients[0]],
    'origin_latitude':[coefficients[1]],
    'destination_longtude':[coefficients[2]],
    'destination_latitude':[coefficients[-1]] })

# Converting this to a spark dataframe
spark_df = spark.createDataFrame(new_table)

spark_df.write.format('delta').mode('overwrite').saveAsTable('ml_coefficients')

StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 134, Finished, Available, Finished)

In [127]:
coefficients[-1]

StatementMeta(, 2517ffbb-de4d-4cb6-a52b-d92a14e34886, 129, Finished, Available, Finished)

-6.769880228464156